# Challenge 1

Who are the top 3 most profiting authors?

In [1]:
import pandas as pd

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db

## Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

- Title ID
- Author ID
- Royalty of each sale for each author

The formula is:
sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
Note that titles.royalty and titleauthor.royaltyper are divided by 100 respectively because they are percentage numbers instead of floats.

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [4]:
%%sql 
select  t.title_id, ta.au_id, (t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


title_id,au_id,sales_royalty
PS3333,172-32-1176,29.984999999999996
BU1032,213-46-8915,3.9979999999999993
BU1032,213-46-8915,7.995999999999999
BU2075,213-46-8915,25.116000000000003
PC1035,238-95-7766,110.16
BU1111,267-41-2394,11.95
TC7777,267-41-2394,8.994
BU7832,274-80-9391,29.984999999999996
BU1032,409-56-7008,5.996999999999999
BU1032,409-56-7008,11.993999999999998


## Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

- Title ID
- Author ID
- Aggregated royalties of each title for each author

Hint: use the SUM subquery and group by both au_id and title_id

In the output of this step, each title should appear only once for each author.

In [5]:
%%sql 
select t.title_id, ta.au_id, sum(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as total_sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id
group by t.title_id, ta.au_id

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


title_id,au_id,total_sales_royalty
BU1032,213-46-8915,11.993999999999998
BU1032,409-56-7008,17.990999999999996
BU1111,267-41-2394,11.95
BU1111,724-80-9391,17.925
BU2075,213-46-8915,25.116000000000003
BU7832,274-80-9391,29.984999999999996
MC2222,712-45-1867,23.987999999999996
MC3021,722-51-5454,21.528
MC3021,899-46-2035,7.176
PC1035,238-95-7766,110.16


## Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

- Author ID
- Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [6]:
%%sql
with tabla_auxiliar as
(select t.title_id, ta.au_id, sum(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as total_sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id
group by t.title_id, ta.au_id)

select au_id, (total_sales_royalty + t.advance) as profits
from tabla_auxiliar aux inner join titles t on aux.title_id = t.title_id
order by profits desc
limit 3

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


au_id,profits
722-51-5454,15021.528
899-46-2035,15007.176
213-46-8915,10150.116


# Challenge 2

Elevating from your solution in Challenge 1, create a table named most_profiting_authors to hold the data about the most profiting authors. The table should have 2 columns:

- au_id - Author ID
- profits - The profits of the author aggregating the advances and royalties

In [7]:
%%sql
CREATE TABLE most_profiting_authors (
  au_id VARCHAR(11) PRIMARY KEY,
  profits DECIMAL NOT NULL
)

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [8]:
%%sql
with tabla_auxiliar2 as 
(with tabla_auxiliar as
(select t.title_id, ta.au_id, sum(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as total_sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id
group by t.title_id, ta.au_id)

select au_id, (total_sales_royalty + t.advance) as profits
from tabla_auxiliar aux inner join titles t on aux.title_id = t.title_id
order by profits desc
limit 3)

insert into most_profiting_authors (au_id, profits)
select au_id, profits from tabla_auxiliar2

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [9]:
%%sql
select * from most_profiting_authors

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


au_id,profits
722-51-5454,15021.528
899-46-2035,15007.176
213-46-8915,10150.116


In [ ]:
# dudas: por qué me saltan errores ResourceClosedError? Por qué no puedo hacerlo con este código?: 

In [12]:
%%sql

with tabla_auxiliar2 as 
(with tabla_auxiliar as
(select t.title_id, ta.au_id, sum(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as total_sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id
group by t.title_id, ta.au_id)

select au_id, (total_sales_royalty + t.advance) as profits
from tabla_auxiliar aux inner join titles t on aux.title_id = t.title_id
order by profits desc
limit 3)

create table most_profiting_authors2 as
select * from tabla_auxiliar2

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
(sqlite3.OperationalError) near "create": syntax error
[SQL: with tabla_auxiliar2 as (with tabla_auxiliar as
(select t.title_id, ta.au_id, sum(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as total_sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id
group by t.title_id, ta.au_id)

select au_id, (total_sales_royalty + t.advance) as profits
from tabla_auxiliar aux inner join titles t on aux.title_id = t.title_id
order by profits desc
limit 3)

create table most_profiting_authors2 as
select * from tabla_auxiliar2]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [14]:
# no es un problema de la query, porque funciona con select: 

In [13]:
%%sql
with tabla_auxiliar2 as 
(with tabla_auxiliar as
(select t.title_id, ta.au_id, sum(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) as total_sales_royalty
from titles t inner join titleauthor ta on t.title_id = ta.title_id
inner join sales s on s.title_id = t.title_id
group by t.title_id, ta.au_id)

select au_id, (total_sales_royalty + t.advance) as profits
from tabla_auxiliar aux inner join titles t on aux.title_id = t.title_id
order by profits desc
limit 3)

select * from tabla_auxiliar2

 * sqlite:///C:\\Users\\Júlia\\Documents\\Ironhack\\1_modulo_1\\Labs\\20-lab-sql-select\\publications.db
Done.


au_id,profits
722-51-5454,15021.528
899-46-2035,15007.176
213-46-8915,10150.116
